1. Momentos
Los momentos de imagen le ayudan a calcular algunas características como el centro de masa del objeto, el área del objeto, etc. Consulte la página de Wikipedia sobre Momentos de imagen.

La función cv.moments() proporciona un diccionario de todos los valores de momento calculados. Vea abajo:

In [ ]:
import numpy as np
import cv2 as cv
img = cv.imread('star.jpg', cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
ret,thresh = cv.threshold(img,127,255,0)
contours,hierarchy = cv.findContours(thresh, 1, 2)
cnt = contours[0]
M = cv.moments(cnt)
print( M )

In [ ]:
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])

2. Área de contorno
El área del contorno viene dada por la función cv.contourArea() o por momentos, M['m00'] .

In [ ]:
area = cv.contourArea(cnt)


3. Perímetro del contorno
También se le llama longitud de arco. Se puede descubrir utilizando la función cv.arcLength() . El segundo argumento especifica si la forma es un contorno cerrado (si se pasa como Verdadero) o simplemente una curva.

In [ ]:
perimeter = cv.arcLength(cnt,True)


4. Aproximación del contorno
Aproxima una forma de contorno a otra forma con menor número de vértices dependiendo de la precisión que especifiquemos. Es una implementación del algoritmo de Douglas-Peucker . Consulte la página de Wikipedia para ver el algoritmo y la demostración.

Para entender esto, suponga que está tratando de encontrar un cuadrado en una imagen, pero debido a algunos problemas en la imagen, no obtuvo un cuadrado perfecto, sino una "mala forma" (como se muestra en la primera imagen a continuación). Ahora puedes usar esta función para aproximar la forma. En esto, el segundo argumento se llama épsilon, que es la distancia máxima desde el contorno hasta el contorno aproximado. Es un parámetro de precisión. Se necesita una selección inteligente de épsilon para obtener el resultado correcto.

In [ ]:
epsilon = 0.1*cv.arcLength(cnt,True)
approx = cv.approxPolyDP(cnt,epsilon,True)

5. Casco convexo
Convex Hull se verá similar a la aproximación de contorno, pero no lo es (ambos pueden proporcionar los mismos resultados en algunos casos). Aquí, la función cv.convexHull() comprueba una curva en busca de defectos de convexidad y la corrige. En términos generales, las curvas convexas son curvas que siempre están abultadas o al menos planas. Y si está abultado por dentro se llama defectos de convexidad. Por ejemplo, consulte la siguiente imagen de la mano. La línea roja muestra el casco convexo de la mano. Las marcas de flecha de doble cara muestran los defectos de convexidad, que son las desviaciones máximas locales del casco respecto de los contornos.

In [ ]:
hull = cv.convexHull(points[, hull[, clockwise[, returnPoints]]])


Los puntos son los contornos por los que pasamos.
El casco es la salida, normalmente lo evitamos.
en el sentido de las agujas del reloj : bandera de orientación. Si es Verdadero, el casco convexo de salida está orientado en el sentido de las agujas del reloj. De lo contrario, se orienta en el sentido contrario a las agujas del reloj.
returnPoints : De forma predeterminada, Verdadero. Luego devuelve las coordenadas de los puntos del casco. Si es False, devuelve los índices de los puntos de contorno correspondientes a los puntos del casco.

In [ ]:
hull = cv.convexHull(cnt)


6. Comprobando la convexidad
Existe una función para comprobar si una curva es convexa o no, cv.isContourConvex() . Simplemente devuelve ya sea Verdadero o Falso. No es un gran trato.

In [ ]:
k = cv.isContourConvex(cnt)


7. Rectángulo delimitador
Hay dos tipos de rectángulos delimitadores.

7.a. Rectángulo delimitador recto
Es un rectángulo recto, no considera la rotación del objeto. Entonces el área del rectángulo delimitador no será mínima. Se encuentra mediante la función cv.boundingRect() .

Sea (x,y) la coordenada superior izquierda del rectángulo y (w,h) su ancho y alto.

In [ ]:
x,y,w,h = cv.boundingRect(cnt)
cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

7.b. Rectángulo girado
Aquí, el rectángulo delimitador se dibuja con un área mínima, por lo que también considera la rotación. La función utilizada es cv.minAreaRect() . Devuelve una estructura Box2D que contiene los siguientes detalles: (centro (x,y), (ancho, alto), ángulo de rotación). Pero para dibujar este rectángulo, necesitamos 4 esquinas del rectángulo. Se obtiene mediante la función cv.boxPoints()

In [ ]:
rect = cv.minAreaRect(cnt)
box = cv.boxPoints(rect)
box = np.int0(box)
cv.drawContours(img,[box],0,(0,0,255),2)

8. Círculo envolvente mínimo
A continuación encontramos el círculo circunstante de un objeto usando la función cv.minEnclosingCircle() . Es un círculo que cubre completamente el objeto con un área mínima.

In [ ]:
(x,y),radius = cv.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
cv.circle(img,center,radius,(0,255,0),2)

9. Ajustar una elipse
El siguiente es ajustar una elipse a un objeto. Devuelve el rectángulo rotado en el que está inscrita la elipse.

In [ ]:
ellipse = cv.fitEllipse(cnt)
cv.ellipse(img,ellipse,(0,255,0),2)

10. Colocar una línea
De manera similar podemos ajustar una línea a un conjunto de puntos. La imagen de abajo contiene un conjunto de puntos blancos. Podemos aproximarle una línea recta

In [ ]:
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(cnt, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)